In [19]:
# -*- coding: utf-8 -*-
# @Time    : 6/11/21 12:57 AM
# @Author  : Yuan Gong
# @Affiliation  : Massachusetts Institute of Technology
# @Email   : yuangong@mit.edu
# @File    : run.py

import argparse
import os
import ast
import pickle
import sys
import time
import torch

In [20]:
os.getcwd()

'/Users/zmoad/Documents/Course/06.AST_Audio/AST/src'

In [21]:
from torch.utils.data import WeightedRandomSampler
basepath = os.path.dirname(os.path.dirname(sys.path[0]))
sys.path.append(basepath)

In [22]:
import sys
sys.path

['/Users/zmoad/Documents/Course/06.AST_Audio/AST/src',
 '/Users/zmoad/opt/miniconda3/lib/python39.zip',
 '/Users/zmoad/opt/miniconda3/lib/python3.9',
 '/Users/zmoad/opt/miniconda3/lib/python3.9/lib-dynload',
 '',
 '/Users/zmoad/Documents/Course/06.AST_Audio/AST/venvast/lib/python3.9/site-packages',
 '/Users/zmoad/Documents/Course/06.AST_Audio/AST/venvast/lib/python3.9/site-packages/IPython/extensions',
 '/Users/zmoad/.ipython',
 '/Users/zmoad/Documents/Course/06.AST_Audio',
 '/Users/zmoad/Documents/Course/06.AST_Audio',
 '/Users/zmoad/Documents/Course/06.AST_Audio']

In [39]:
import dataloader
import models
import numpy as np
from traintest import train, validate

In [41]:
print("I am process %s, running on %s: starting (%s)" % (os.getpid(), os.uname()[1], time.asctime()))

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--data-train", type=str, default='', help="training data json")
parser.add_argument("--data-val", type=str, default='', help="validation data json")
parser.add_argument("--data-eval", type=str, default='', help="evaluation data json")
parser.add_argument("--label-csv", type=str, default='', help="csv with class labels")
parser.add_argument("--n_class", type=int, default=527, help="number of classes")
# parser.add_argument("--model", type=str, default='ast', help="the model used")
# parser.add_argument("--dataset", type=str, default="audioset", help="the dataset used", choices=["audioset", "esc50", "speechcommands"])

# parser.add_argument("--exp-dir", type=str, default="", help="directory to dump experiments")
# parser.add_argument('--lr', '--learning-rate', default=0.001, type=float, metavar='LR', help='initial learning rate')
# parser.add_argument("--optim", type=str, default="adam", help="training optimizer", choices=["sgd", "adam"])
# parser.add_argument('-b', '--batch-size', default=12, type=int, metavar='N', help='mini-batch size')
# parser.add_argument('-w', '--num-workers', default=32, type=int, metavar='NW', help='# of workers for dataloading (default: 32)')
# parser.add_argument("--n-epochs", type=int, default=1, help="number of maximum training epochs")
# # not used in the formal experiments
# parser.add_argument("--lr_patience", type=int, default=2, help="how many epoch to wait to reduce lr if mAP doesn't improve")

# parser.add_argument("--n-print-steps", type=int, default=100, help="number of steps to print statistics")
# parser.add_argument('--save_model', help='save the model or not', type=ast.literal_eval)

# parser.add_argument('--freqm', help='frequency mask max length', type=int, default=0)
# parser.add_argument('--timem', help='time mask max length', type=int, default=0)
# parser.add_argument("--mixup", type=float, default=0, help="how many (0-1) samples need to be mixup during training")
# parser.add_argument("--bal", type=str, default=None, help="use balanced sampling or not")
# # the stride used in patch spliting, e.g., for patch size 16*16, a stride of 16 means no overlapping, a stride of 10 means overlap of 6.
# parser.add_argument("--fstride", type=int, default=10, help="soft split freq stride, overlap=patch_size-stride")
# parser.add_argument("--tstride", type=int, default=10, help="soft split time stride, overlap=patch_size-stride")
# parser.add_argument('--imagenet_pretrain', help='if use ImageNet pretrained audio spectrogram transformer model', type=ast.literal_eval, default='True')
# parser.add_argument('--audioset_pretrain', help='if use ImageNet and audioset pretrained audio spectrogram transformer model', type=ast.literal_eval, default='False')

# args = parser.parse_args(args=['--data-train','--data-val','--data-eval','--label-csv','--n_class'])
# print(args)

I am process 19405, running on ZongchaodeiMac.local: starting (Thu Nov 11 21:41:09 2021)


_StoreAction(option_strings=['--n_class'], dest='n_class', nargs=None, const=None, default=527, type=<class 'int'>, choices=None, help='number of classes', metavar=None)

In [33]:
# transformer based model
# if args.model == 'ast':
print('now train a audio spectrogram transformer model')
# dataset spectrogram mean and std, used to normalize the input
norm_stats = {'audioset':[-4.2677393, 4.5689974], 'esc50':[-6.6268077, 5.358466], 'speechcommands':[-6.845978, 5.5654526]}
target_length = {'audioset':1024, 'esc50':512, 'speechcommands':128}

# if add noise for data augmentation, only use for speech commands
noise = {'audioset': False, 'esc50': False, 'speechcommands':True}

now train a audio spectrogram transformer model


In [27]:
# audio_conf = {'num_mel_bins': 128, 'target_length': target_length[args.dataset],
#               'freqm': args.freqm, 'timem': args.timem, 'mixup': args.mixup,
#               'dataset': args.dataset, 'mode':'train',
#               'mean':norm_stats[args.dataset][0], 'std':norm_stats[args.dataset][1],
#               'noise':noise[args.dataset]}
# val_audio_conf = {'num_mel_bins': 128, 'target_length': target_length[args.dataset],
#                   'freqm': 0, 'timem': 0, 'mixup': 0, 'dataset': args.dataset,
#                   'mode':'evaluation', 'mean':norm_stats[args.dataset][0], 
#                   'std':norm_stats[args.dataset][1], 'noise':False}

audio_conf = {'num_mel_bins': 128, 'target_length': target_length[args.dataset],
              'freqm': 0, 'timem': 0, 'mixup': 0,
              'dataset': args.dataset, 'mode':'train',
              'mean':norm_stats[args.dataset][0], 'std':norm_stats[args.dataset][1],
              'noise':noise[args.dataset]}
val_audio_conf = {'num_mel_bins': 128, 'target_length': target_length[args.dataset],
                  'freqm': 0, 'timem': 0, 'mixup': 0, 'dataset': args.dataset,
                  'mode':'evaluation', 'mean':norm_stats[args.dataset][0], 
                  'std':norm_stats[args.dataset][1], 'noise':False}

NameError: name 'args' is not defined

In [28]:
# if args.bal == 'bal':
#     print('balanced sampler is being used')
#     samples_weight = np.loadtxt(args.data_train[:-5]+'_weight.csv', delimiter=',')
#     sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)

#     train_loader = torch.utils.data.DataLoader(
#         dataloader.AudiosetDataset(args.data_train, label_csv=args.label_csv, audio_conf=audio_conf),
#         batch_size=args.batch_size, sampler=sampler, num_workers=args.num_workers, pin_memory=True)
# else:

print('balanced sampler is not used')
train_loader = torch.utils.data.DataLoader(
    dataloader.AudiosetDataset(args.data_train, label_csv=args.label_csv, audio_conf=audio_conf),
    batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, pin_memory=True)



balanced sampler is not used


NameError: name 'args' is not defined

In [ ]:
val_loader = torch.utils.data.DataLoader(
    dataloader.AudiosetDataset(args.data_val, label_csv=args.label_csv, audio_conf=val_audio_conf),
    batch_size=args.batch_size*2, shuffle=False, num_workers=args.num_workers, pin_memory=True)

In [ ]:
audio_model = models.ASTModel(label_dim=args.n_class, fstride=args.fstride, tstride=args.tstride, input_fdim=128,
                              input_tdim=target_length[args.dataset], imagenet_pretrain=args.imagenet_pretrain,
                              audioset_pretrain=args.audioset_pretrain, model_size='base384')



In [ ]:
print("\nCreating experiment directory: %s" % args.exp_dir)
os.makedirs("%s/models" % args.exp_dir)
with open("%s/args.pkl" % args.exp_dir, "wb") as f:
    pickle.dump(args, f)

    
print('Now starting training for {:d} epochs'.format(args.n_epochs))
train(audio_model, train_loader, val_loader, args)# 这个train的话，当然是包括“十折交叉里面的训练数据集+验证数据集（train_loader）” + testing数据集（val_loader）

In [ ]:
# for speechcommands dataset, evaluate the best model on validation set on the test set
# 后面如果涉及到评估的时候，可以用这部分的代码

# if args.dataset == 'speechcommands':
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     sd = torch.load(args.exp_dir + '/models/best_audio_model.pth', map_location=device)
#     audio_model = torch.nn.DataParallel(audio_model)
#     audio_model.load_state_dict(sd)


#     # best model on the validation set # 内部的验证数据集
#     stats, _ = validate(audio_model, val_loader, args, 'valid_set')
#     # note it is NOT mean of class-wise accuracy
#     val_acc = stats[0]['acc']
#     val_mAUC = np.mean([stat['auc'] for stat in stats])
#     print('---------------evaluate on the validation set---------------')
#     print("Accuracy: {:.6f}".format(val_acc))
#     print("AUC: {:.6f}".format(val_mAUC))


#     # test the model on the evaluation set # 独立的第三方数据集，叫做evaluation set
#     eval_loader = torch.utils.data.DataLoader(
#         dataloader.AudiosetDataset(args.data_eval, label_csv=args.label_csv, audio_conf=val_audio_conf),
#         batch_size=args.batch_size*2, shuffle=False, num_workers=args.num_workers, pin_memory=True)
      # 从这个数据来看，input就是音频文件的路径，以及具体的分类标签
    
    
#     stats, _ = validate(audio_model, eval_loader, args, 'eval_set')
#     eval_acc = stats[0]['acc']
#     eval_mAUC = np.mean([stat['auc'] for stat in stats])


#     print('---------------evaluate on the test set---------------')
#     print("Accuracy: {:.6f}".format(eval_acc))
#     print("AUC: {:.6f}".format(eval_mAUC))
#     np.savetxt(args.exp_dir + '/eval_result.csv', [val_acc, val_mAUC, eval_acc, eval_mAUC])